In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

# df = pd.read_feather("traning_imgs.df")
# old = pd.read_feather("../data_analysis/prelim.feather").set_index("key")
# df["seq_key"] = old.loc[df["key"].values,"sequence_key"].values
# df["view_direction"] = old.loc[df["key"].values,"view_direction"].values
# df["city"] = df.path.apply(lambda x: x[22: x.find("/",22)])
# func = lambda s: s[s.find("images/")+7:]
# df["local_path"] =  "../seq_data/" + df.ds_type + "/" + df.path.apply(func).values
# df = df.sort_values(["city","seq_key","og_cluster"])
# df.index = range(len(df))
# df.to_feather("traning_imgs.df")

In [37]:
df = pd.read_feather("traning_imgs.df")

(df.view_direction=="Sideways").apply(int).mean()

0.3235687228603985

In [39]:
ds = tf.data.Dataset.from_tensor_slices((df.local_path.values,(df.view_direction=="Sideways").apply(int).values))
model = tf.keras.models.load_model()

In [9]:
len("Places_Data/train_val/")

22

In [7]:

paths

0         ../seq_data/train/KYBNnSy0RySIkI1YNkGC1w.jpg
1         ../seq_data/train/3uz3XZ4EHqurWW9D04u62A.jpg
2         ../seq_data/train/7bZHPrh4efxZJrleUQHZQg.jpg
3         ../seq_data/train/RQmQk4CFgl16KSMqpEpvlA.jpg
4         ../seq_data/train/zWIH_aORFE4Gn1sS9E9d9Q.jpg
                              ...                     
162350     ../seq_data/test/Sjnj9MYJlZRRAQnd0eP47r.jpg
162351     ../seq_data/test/BzgzU8TWvAJSF6zha8-Z1_.jpg
162352     ../seq_data/test/f8xCd8phDohvCcOb3xEyDL.jpg
162353     ../seq_data/test/crdg_zMtmLo2TnwpQ-vWTz.jpg
162354     ../seq_data/test/T9oQ9d0vh0VgWFE_M0nxSu.jpg
Name: ds_type, Length: 162355, dtype: object